## Simple Gen AI APP Using Langchain

In [36]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [37]:
## Data Ingestion From The Website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://python.langchain.com/docs/tutorials/rag/")
loader

In [38]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/rag/', 'title': 'Build a Retrieval Augmented Generation (RAG) App | 🦜️🔗 LangChain', 'description': 'One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or RAG.', 'language': 'en'}, page_content='\n\n\n\n\nBuild a Retrieval Augmented Generation (RAG) App | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM Application with LCELBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Extraction ChainBuild an AgentTaggingdata_generationBuild a Local RAG Applic

### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB


In [39]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(docs)

In [40]:
documents

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/rag/', 'title': 'Build a Retrieval Augmented Generation (RAG) App | 🦜️🔗 LangChain', 'description': 'One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or RAG.', 'language': 'en'}, page_content='Build a Retrieval Augmented Generation (RAG) App | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/rag/', 'title': 'Build a Retrieval Augmented Generation (RAG) App | 🦜️🔗 LangChain', 'description': 'One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or RAG.

In [41]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


In [42]:
from langchain_community.vectorstores import FAISS

vectorstoredb = FAISS.from_documents(documents,embeddings)

In [43]:
vectorstoredb

In [44]:
query = 'how to do indexing spliting?'

result = vectorstoredb.similarity_search(query)

result

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/rag/', 'title': 'Build a Retrieval Augmented Generation (RAG) App | 🦜️🔗 LangChain', 'description': 'One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or RAG.', 'language': 'en'}, page_content='2. Indexing: Split\u200b\nOur loaded document is over 42k characters long. This is too long to fit\nin the context window of many models. Even for those models that could\nfit the full post in their context window, models can struggle to find\ninformation in very long inputs.\nTo handle this we’ll split the Document into chunks for embedding and\nvector storage. This should help us retrieve only the most relevant bits\nof the blog post at run time.\nIn this case we’ll split our documents into chunks of 1000 cha

In [45]:
for chunk in result:
    print(chunk.page_content)

2. Indexing: Split​
Our loaded document is over 42k characters long. This is too long to fit
in the context window of many models. Even for those models that could
fit the full post in their context window, models can struggle to find
information in very long inputs.
To handle this we’ll split the Document into chunks for embedding and
vector storage. This should help us retrieve only the most relevant bits
of the blog post at run time.
In this case we’ll split our documents into chunks of 1000 characters
with 200 characters of overlap between chunks. The overlap helps
mitigate the possibility of separating a statement from important
context related to it. We use the
RecursiveCharacterTextSplitter,
which will recursively split the document using common separators like
new lines until each chunk is the appropriate size. This is the
recommended text splitter for generic text use cases.
We set add_start_index=True so that the character index at which each
3. Indexing: Store​
Now we need t

In [46]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [47]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    """
)

document_chain =  create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7d85175fad80>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7d85175f92e0>, root_client=<openai.OpenAI object at 0x7d85175f9d30>, root_async_client=<openai.AsyncOpenAI object at 0x7d85175fa840>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, conf

In [48]:
# from langchain_core.documents import Document

# document_chain.invoke({
#     "input":"how to do indexing spliting?",
#     "context":[Document(page_content='2. Indexing: Split\u200b\nOur loaded document is over 42k characters long. This is too long to fit\nin the context window of many models. Even for those models that could\nfit the full post in their context window, models can struggle to find\ninformation in very long inputs.\nTo handle this we’ll split the Document into chunks for embedding and\nvector storage. This should help us retrieve only the most relevant bits\nof the blog post at run time.\nIn this case we’ll split our documents into chunks of 1000 characters\nwith 200 characters of overlap between chunks. The overlap helps\nmitigate the possibility of separating a statement from important\ncontext related to it. We use the\nRecursiveCharacterTextSplitter,\nwhich will recursively split the document using common separators like\nnew lines until each chunk is the appropriate size. This is the\nrecommended text splitter for generic text use cases.\nWe set add_start_index=True so that the character index at which each')]
# })

In [49]:
retriver = vectorstoredb.as_retriever()

from langchain.chains import create_retrieval_chain 
retrival_chain = create_retrieval_chain(retriver, )

retrival_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7d8535593da0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n    {context}\n    </context>\n    '), additional_kwargs={})])
  

In [50]:
res =  retrival_chain.invoke({"input":"what is RAG?"})

In [51]:
res

{'input': 'what is RAG?',
 'context': [Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/rag/', 'title': 'Build a Retrieval Augmented Generation (RAG) App | 🦜️🔗 LangChain', 'description': 'One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or RAG.', 'language': 'en'}, page_content="One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or RAG.\nThis tutorial will show how to build a simple Q&A application\nover a text data source. Along the way we’ll go over a typical Q&A\narchitecture and highlight additional resources for more advanced Q&A te

In [54]:
print(res['answer'])

RAG stands for Retrieval Augmented Generation, which is a technique for augmenting LLM (Large Language Model) knowledge with additional data. It involves bringing the appropriate information and inserting it into the model prompt to enable AI applications to reason about private data or data introduced after a model's training cutoff date.
